Setup Libraries

In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split,ConcatDataset
from sklearn.metrics import f1_score
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict
from collections import Counter

Datasets Path

In [3]:
data_dir = 'C:/Users/CaioGabrielAdernedeM/OneDrive/IPB/ipb_sistemas_inteligentes/projeto_final/train'
eval_dir = 'C:/Users/CaioGabrielAdernedeM/OneDrive/IPB/ipb_sistemas_inteligentes/projeto_final/evaluation_set'

Load and Split Train Dataset

In [5]:

# Define transformations (resize, normalize, etc.)
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize all images to 224x224
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load dataset

dataset = datasets.ImageFolder(root=data_dir, transform=transform)

# Split dataset into training and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Calculate split sizes
dataset_size = len(dataset)
train_size = int(0.8 * dataset_size)
val_size = int(0.1 * dataset_size)
test_size = dataset_size - train_size - val_size  # Ensure all samples are used

# Split the dataset
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Print dataset sizes
print(f"Total dataset size: {dataset_size}")
print(f"Training set size: {len(train_dataset)}")
print(f"Validation set size: {len(val_dataset)}")
print(f"Test set size: {len(test_dataset)}")

# Function to count labels in a dataset
def count_labels(subset):
    labels = [dataset.targets[idx] for idx in subset.indices]  # Get labels for the subset
    label_counts = Counter(labels)
    return {dataset.classes[label]: count for label, count in label_counts.items()}

# Print label counts for each subset
print("Training set label counts:", count_labels(train_dataset))
print("Validation set label counts:", count_labels(val_dataset))
print("Test set label counts:", count_labels(test_dataset))

# Create DataLoaders (optional, if needed for training/validation/testing)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


Total dataset size: 251
Training set size: 200
Validation set size: 25
Test set size: 26
Training set label counts: {'positive': 86, 'negative': 114}
Validation set label counts: {'negative': 15, 'positive': 10}
Test set label counts: {'negative': 11, 'positive': 15}


In [8]:
# Define a simple feedforward neural network
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten the input
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Define hyperparameters for each model
models_config = [
    {"hidden_size": 64, "learning_rate": 0.001},
    {"hidden_size": 128, "learning_rate": 0.001},
    {"hidden_size": 256, "learning_rate": 0.001},
    {"hidden_size": 64, "learning_rate": 0.0005},
    {"hidden_size": 128, "learning_rate": 0.0005},
    {"hidden_size": 256, "learning_rate": 0.0005},
    {"hidden_size": 64, "learning_rate": 0.0001},
    {"hidden_size": 128, "learning_rate": 0.0001},
    {"hidden_size": 256, "learning_rate": 0.0001}
]

# Function to train and validate a model
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=20):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        # Validation step
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        
        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {running_loss/len(train_loader):.4f}, "
              f"Val Loss: {val_loss/len(val_loader):.4f}, Val Accuracy: {100 * correct / total:.2f}%")

# DataFrame to store results
results = []

# Train and evaluate each model
input_size = 224 * 224 * 3  # Input size based on image dimensions
output_size = len(dataset.classes)  # Number of classes

for i, config in enumerate(models_config):
    print(f"\nTraining Model {i+1} with Hidden Size: {config['hidden_size']}, Learning Rate: {config['learning_rate']}")
    
    # Initialize model, loss function, and optimizer
    model = SimpleNN(input_size=input_size, hidden_size=config['hidden_size'], output_size=output_size)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=config['learning_rate'])
    
    # Train the model
    train_model(model, train_loader, val_loader, criterion, optimizer, epochs=20)

    # Test the model
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to("cuda" if torch.cuda.is_available() else "cpu"), labels.to("cuda" if torch.cuda.is_available() else "cpu")
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    test_accuracy = 100 * correct / total
    print(f"Model {i+1} Test Accuracy: {test_accuracy:.2f}%")

    # Append results to DataFrame
    results.append({
        "Model Number": i+1,
        "Hidden Size": config['hidden_size'],
        "Learning Rate": config['learning_rate'],
        "Test Accuracy": test_accuracy
    })

# Create DataFrame and save results
results_df = pd.DataFrame(results)
print("\nFinal Results:")
print(results_df)

# Save results to CSV
results_df.to_csv("model_results.csv", index=False)


Training Model 1 with Hidden Size: 64, Learning Rate: 0.001
Epoch 1/20, Train Loss: 10.6185, Val Loss: 0.0000, Val Accuracy: 100.00%
Epoch 2/20, Train Loss: 2.8479, Val Loss: 0.0000, Val Accuracy: 100.00%
Epoch 3/20, Train Loss: 2.2607, Val Loss: 0.0000, Val Accuracy: 100.00%
Epoch 4/20, Train Loss: 0.5789, Val Loss: 0.0000, Val Accuracy: 100.00%
Epoch 5/20, Train Loss: 0.3684, Val Loss: 0.0000, Val Accuracy: 100.00%
Epoch 6/20, Train Loss: 0.2234, Val Loss: 0.1497, Val Accuracy: 92.00%
Epoch 7/20, Train Loss: 0.1515, Val Loss: 0.0001, Val Accuracy: 100.00%
Epoch 8/20, Train Loss: 0.0707, Val Loss: 0.0000, Val Accuracy: 100.00%
Epoch 9/20, Train Loss: 0.0608, Val Loss: 0.0000, Val Accuracy: 100.00%
Epoch 10/20, Train Loss: 0.0007, Val Loss: 0.0001, Val Accuracy: 100.00%
Epoch 11/20, Train Loss: 0.0003, Val Loss: 0.0001, Val Accuracy: 100.00%
Epoch 12/20, Train Loss: 0.0000, Val Loss: 0.0002, Val Accuracy: 100.00%
Epoch 13/20, Train Loss: 0.0000, Val Loss: 0.0002, Val Accuracy: 100.00%

KeyboardInterrupt: 

In [9]:
# Combine training and validation datasets
combined_train_dataset = ConcatDataset([train_dataset, val_dataset])

# Create DataLoaders
combined_train_loader = DataLoader(combined_train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Hyperparameters
params = {"hidden_size": 256, "learning_rate": 0.0005, "epochs": 20}
input_size = 224 * 224 * 3  # Input size based on image dimensions
output_size = len(dataset.classes)  # Number of classes

# Initialize model, loss function, and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleNN(input_size=input_size, hidden_size=params["hidden_size"], output_size=output_size).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=params["learning_rate"])

# Training the model
def train_model(model, train_loader, criterion, optimizer, epochs):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}")

# Train the model
print("Training the model with the combined training and validation datasets...")
train_model(model, combined_train_loader, criterion, optimizer, epochs=params["epochs"])

# Testing the model
def test_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")
    return accuracy

# Test the model
print("Testing the model with the test dataset...")
test_accuracy = test_model(model, test_loader)

# Save results to a DataFrame
results = pd.DataFrame([{
    "Hidden Size": params["hidden_size"],
    "Learning Rate": params["learning_rate"],
    "Epochs": params["epochs"],
    "Test Accuracy": test_accuracy
}])

print("\nFinal Results:")
print(results)

# Save the results to a CSV file
results.to_csv("final_model_results.csv", index=False)

Training the model with the combined training and validation datasets...
Epoch 1/20, Loss: 7.0922
Epoch 2/20, Loss: 4.3611
Epoch 3/20, Loss: 1.4017
Epoch 4/20, Loss: 0.5902
Epoch 5/20, Loss: 0.4588
Epoch 6/20, Loss: 0.2705
Epoch 7/20, Loss: 0.2636
Epoch 8/20, Loss: 0.3637
Epoch 9/20, Loss: 0.1390
Epoch 10/20, Loss: 0.0557
Epoch 11/20, Loss: 0.0130
Epoch 12/20, Loss: 0.0748
Epoch 13/20, Loss: 0.0592
Epoch 14/20, Loss: 0.0733
Epoch 15/20, Loss: 0.1436
Epoch 16/20, Loss: 0.0000
Epoch 17/20, Loss: 0.0628
Epoch 18/20, Loss: 0.0000
Epoch 19/20, Loss: 0.0146
Epoch 20/20, Loss: 0.0000
Testing the model with the test dataset...
Test Accuracy: 100.00%

Final Results:
   Hidden Size  Learning Rate  Epochs  Test Accuracy
0          256         0.0005      20          100.0
